In [ ]:
# ! git clone https://github.com/huggingface/diffusers && cd diffusers && pip install .

In [ ]:
# ! cd diffusers/examples/text_to_image  && pip install -r requirements.txt

In [3]:
# !  git config --global credential.helper store && huggingface-cli login

In [ ]:
# ! accelerate launch --mixed_precision="fp16"  train_text_to_image.py \
#   --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1" \
#   --dataset_name="mintz1104/diffusion_stage_design_japanese_anime_style" \
#   --use_ema \
#   --resolution=768 --center_crop --random_flip \
#   --train_batch_size=1 \
#   --gradient_accumulation_steps=4 \
#   --gradient_checkpointing \
#   --max_train_steps=15000 \
#   --learning_rate=1e-05 \
#   --max_grad_norm=1 \
#   --lr_scheduler="constant" --lr_warmup_steps=0 \
#   --output_dir="tuned-model"

In [10]:
# import os
# from datasets import Dataset, DatasetDict, Features, Value, Image


# def load_drones_dataset(image_folder: str) -> DatasetDict:
#     """
#     Загружает изображения и возвращает DatasetDict с набором train.

#     Args:
#         image_folder (str): Путь к папке с изображениями.

#     Returns:
#         DatasetDict: Датасет с разделением на train.
#     """
#     image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')]

#     data = {
#         "image": [],
#         "text": []
#     }

#     for idx, image_file in enumerate(image_files):
#         image_path = os.path.join(image_folder, image_file)

#         # Генерируем подпись как имя файла без расширения
#         caption = os.path.splitext(image_file)[0]

#         data["image"].append(image_path)
#         data["text"].append(caption)

#     # Определяем структуру данных (features) для корректного хранения
#     features = Features({
#         "image": Image(),  # Автоматическая загрузка изображений
#         "text": Value("string")  # Строка
#     })

#     # Создаем Dataset
#     dataset = Dataset.from_dict(data, features=features)

#     # Оборачиваем в DatasetDict с подмножеством train
#     dataset_dict = DatasetDict({"train": dataset})

#     return dataset_dict


# def save_dataset(dataset: DatasetDict, output_folder: str):
#     """
#     Сохраняет DatasetDict в указанную папку.

#     Args:
#         dataset (DatasetDict): Датасет, который нужно сохранить.
#         output_folder (str): Путь к папке для сохранения.
#     """
#     os.makedirs(output_folder, exist_ok=True)  # Создаем папку, если её нет
#     dataset.save_to_disk(output_folder)
#     print(f"Датасет сохранён в формате DatasetDict в папку: {output_folder}")

In [ ]:
# from datasets import load_from_disk

# image_folder = "./drone-dataset"  
# output_folder = "./dataset"  

# dataset = load_drones_dataset(image_folder)
# save_dataset(dataset, output_folder)

# dataset_path = "./dataset"
# dataset = load_from_disk(dataset_path)
# print(dataset)


In [22]:
# from datasets import load_dataset

# # Загрузка датасета
# dataset = load_dataset("mintz1104/diffusion_stage_design_japanese_anime_style")


In [1]:
prompts = [
    "drone lightshow of a glowing tree of life with intricate roots and branches, illuminated in green and gold, made with 2000 drones on a black background",
    "drone lightshow of a detailed golden phoenix rising with glowing wings and fire-like trails made with 2000 drones on a black background",
    "drone lightshow of the face of Elon Musk with intricate details made with 2000 drones on a black background",
    "drone lightshow of a red and white heart-shaped pattern over the ocean with reflections of the lights made with 2000 drones",
    "drone lightshow of a golden sun rising behind snow-capped mountains with glowing rays of light made with 2000 drones",
    "drone lightshow of a detailed red and silver motorcycle speeding forward with glowing exhaust made with 1500 drones on a black background",
    "drone lightshow of a bright multicolored butterfly flapping its wings over a field with trees and ambient lighting effects made with 2000 drones on a black background",
    "drone lightshow of a shimmering blue and white globe of the Earth with detailed continents made with 1000 drones on a black background",
    "drone lightshow of a futuristic robot standing tall with glowing blue eyes and moving arms made with 2000 drones on a black background"
]

# Stable-Diffusion 2.1: Fine tuning 

In [ ]:
# ! accelerate launch --mixed_precision="fp16"  diffusers/examples/text_to_image/train_text_to_image.py \
#   --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1" \
#   --train_data_dir="./dataset/" \
#   --resolution=768 \
#   --center_crop \
#   --train_batch_size=2 \
#   --gradient_accumulation_steps=4 \
#   --gradient_checkpointing \
#   --max_train_steps=15000 \
#   --learning_rate=1e-05 \
#   --max_grad_norm=1 \
#   --lr_scheduler="cosine" --lr_warmup_steps=100 \
#   --output_dir="tuned-model-2.1" \
#   --checkpointing_steps=5000

In [ ]:
# from diffusers import StableDiffusionPipeline, UNet2DConditionModel
# import torch

# # # Загрузить модель
# # model_path = "tuned-model-2.1"
# # pipe_tuned = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")

# # Загрузить чекпоинт
# model_path = "stabilityai/stable-diffusion-2-1"
# pipe_tuned = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")
# unet = UNet2DConditionModel.from_pretrained("tuned-model-2.1/checkpoint-15000", subfolder="unet", torch_dtype=torch.float16)
# pipe_tuned.unet = unet
# pipe_tuned.to("cuda")


# from diffusers import EulerAncestralDiscreteScheduler
# pipe_tuned.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe_tuned.scheduler.config)

In [ ]:
# import matplotlib.pyplot as plt
# from PIL import Image

# # Параметры отображения
# desired_height = 768
# desired_width = 768

# # Устанавливаем количество изображений в строке
# images_per_row = 3

# # Создаём фигуру для сетки
# fig, axes = plt.subplots(
#     nrows=(len(prompts) + images_per_row - 1) // images_per_row,  # Расчёт строк
#     ncols=images_per_row,
#     figsize=(15, 15)
# )

# # Генерация изображений
# for i, prompt in enumerate(prompts):
#     ax = axes[i // images_per_row, i % images_per_row]  # Выбираем позицию
#     ax.axis("off")  # Убираем оси
#     ax.set_title(f"Prompt {i + 1}")

#     # Генерация изображения
#     tuned_image = pipe_tuned(
#         prompt=prompt,
#         num_inference_steps=70,
#         guidance_scale=15,
#         random_state=42,
#         height=desired_height,
#         width=desired_width
#     ).images[0]

#     # Уменьшаем размер для отображения
#     small_image = tuned_image.resize((256, 256), Image.Resampling.LANCZOS)
#     ax.imshow(small_image)

# # Убираем лишние оси, если количество изображений не кратно `images_per_row`
# for ax in axes.flat[len(prompts):]:
#     ax.axis("off")

# plt.tight_layout()
# plt.show()


In [ ]:
# prompts = [
#     "drone lightshow of a glowing tree of life with intricate roots and branches, illuminated in green and gold, made with 2000 drones on a black background",
#     "drone lightshow of a detailed golden phoenix rising with glowing wings and fire-like trails made with 2000 drones on a black background",
#     "drone lightshow of the face of Elon Musk with intricate details made with 2000 drones on a black background",
#     "drone lightshow of a red and white heart-shaped pattern over the ocean with reflections of the lights made with 2000 drones",
#     "drone lightshow of a golden sun rising behind snow-capped mountains with glowing rays of light made with 2000 drones",
#     "drone lightshow of a detailed red and silver motorcycle speeding forward with glowing exhaust made with 1500 drones on a black background",
#     "drone lightshow of a bright multicolored butterfly flapping its wings over a field with trees and ambient lighting effects made with 2000 drones on a black background",
#     "drone lightshow of a shimmering blue and white globe of the Earth with detailed continents made with 1000 drones on a black background",
#     "drone lightshow of a futuristic robot standing tall with glowing blue eyes and moving arms made with 2000 drones on a black background"
# ]

# Stable-Diffusion 2.1: Lora

In [ ]:
# ! accelerate launch --mixed_precision="no" diffusers/examples/text_to_image/train_text_to_image_lora.py \
#   --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1" \
#   --train_data_dir="./dataset/" \
#   --caption_column="text" \
#   --resolution=768 --random_flip \
#   --train_batch_size=1 \
#   --num_train_epochs=50 --checkpointing_steps=5000 \
#   --learning_rate=1e-04 \
#   --lr_scheduler="constant" \
#   --lr_warmup_steps=0 \
#   --seed=42 \
#   --output_dir="tuned-model-lora" 
 

In [ ]:
# from diffusers import StableDiffusionPipeline
# import torch

# model_path = "tuned-model-2.1-lora"
# model_name = "stabilityai/stable-diffusion-2-1"
# pipe_tuned = StableDiffusionPipeline.from_pretrained(model_name, torch_dtype=torch.float16)
# pipe_tuned.unet.load_attn_procs(model_path)
# pipe_tuned.to("cuda")


# from diffusers import EulerAncestralDiscreteScheduler
# pipe_tuned.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe_tuned.scheduler.config)

In [ ]:
# import matplotlib.pyplot as plt
# from PIL import Image

# # Параметры отображения
# desired_height = 768
# desired_width = 768

# # Устанавливаем количество изображений в строке
# images_per_row = 3

# # Создаём фигуру для сетки
# fig, axes = plt.subplots(
#     nrows=(len(prompts) + images_per_row - 1) // images_per_row,  # Расчёт строк
#     ncols=images_per_row,
#     figsize=(15, 15)
# )

# # Генерация изображений
# for i, prompt in enumerate(prompts):
#     ax = axes[i // images_per_row, i % images_per_row]  # Выбираем позицию
#     ax.axis("off")  # Убираем оси
#     ax.set_title(f"Prompt {i + 1}")

#     # Генерация изображения
#     tuned_image = pipe_tuned(
#         prompt=prompt,
#         num_inference_steps=70,
#         guidance_scale=6.5,
#         random_state=42,
#         height=desired_height,
#         width=desired_width
#     ).images[0]

#     # Уменьшаем размер для отображения
#     small_image = tuned_image.resize((256, 256), Image.Resampling.LANCZOS)
#     ax.imshow(small_image)

# # Убираем лишние оси, если количество изображений не кратно `images_per_row`
# for ax in axes.flat[len(prompts):]:
#     ax.axis("off")

# plt.tight_layout()
# plt.show()


In [1]:
# prompts = [
#     "drone lightshow of a glowing tree of life with intricate roots and branches, illuminated in green and gold, made with 2000 drones on a black background",
#     "drone lightshow of a detailed golden phoenix rising with glowing wings and fire-like trails made with 2000 drones on a black background",
#     "drone lightshow of face of Elon Musk on a black background",
#     "drone lightshow of a red and white heart-shaped pattern over the ocean with reflections of the lights made with 2000 drones",
#     "drone lightshow of a golden sun rising behind snow-capped mountains with glowing rays of light made with 2000 drones",
#     "drone lightshow of a detailed red and silver motorcycle speeding forward with glowing exhaust made with 1500 drones on a black background",
#     "drone lightshow of a bright multicolored butterfly flapping its wings over a field with trees and ambient lighting effects made with 2000 drones on a black background",
#     "drone lightshow of a shimmering blue and white globe of the Earth with detailed continents made with 1000 drones on a black background",
#     "drone lightshow of a futuristic robot standing tall with glowing blue eyes and moving arms made with 2000 drones on a black background"
# ]

# Stable diffusion 3.x: Fine tuning

In [2]:
# !pip install -r diffusers/examples/dreambooth/requirements_sd3.txt

In [ ]:
! accelerate launch diffusers/examples/dreambooth/train_dreambooth_sd3.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-3.5-medium"  \
  --dataset_name="./dataset/" \
  --output_dir="tuned-model-3.5" \
  --resolution=768 \
  --train_batch_size=2 \
  --gradient_accumulation_steps=1 \
  --learning_rate=5e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --mixed_precision="fp16" \
  --image_column="image" \
  --caption_column="text" \
  --use_8bit_adam \
  --gradient_checkpointing \
  --max_train_steps=6500 \
  --checkpointing_steps=5000 

In [ ]:
# # "stabilityai/stable-diffusion-3.5-large"
# # "stabilityai/stable-diffusion-3.5-medium"
# ! accelerate launch diffusers/examples/dreambooth/train_dreambooth_lora_sd3.py \
#   --pretrained_model_name_or_path="stabilityai/stable-diffusion-3.5-medium"  \
#   --dataset_name="./dataset/" \
#   --output_dir="tuned-model-3.5-medium-lora" \
#   --resolution=512 \
#   --train_batch_size=2 \
#   --gradient_accumulation_steps=1 \
#   --learning_rate=1e-4 \
#   --lr_scheduler="constant" \
#   --lr_warmup_steps=0 \
#   --seed="0" \
#   --image_column="image" \
#   --caption_column="text" \
#   --mixed_precision="fp16" \
#   --use_8bit_adam \
#   --max_train_steps=40000 \
#   --checkpointing_steps=5000 

In [ ]:
from diffusers import StableDiffusion3Pipeline
import torch

model_path = "tuned-model-3.5-medium-lora"
model_name = "stabilityai/stable-diffusion-3.5-medium"


pipe_tuned = StableDiffusion3Pipeline.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")
pipe_tuned.load_lora_weights(model_path)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

desired_height = 768
desired_width = 768
images_per_row = 3

fig, axes = plt.subplots(
    nrows=(len(prompts) + images_per_row - 1) // images_per_row,  # Расчёт строк
    ncols=images_per_row,
    figsize=(15, 15)
)

for i, prompt in enumerate(prompts):
    ax = axes[i // images_per_row, i % images_per_row]  # Выбираем позицию
    ax.axis("off")  # Убираем оси
    ax.set_title(f"Prompt {i + 1}")

    tuned_image = pipe_tuned(
        prompt=prompt,
        num_inference_steps=100,
        guidance_scale=6.5,
        height=desired_height,
        width=desired_width
    ).images[0]

    small_image = tuned_image.resize((256, 256), Image.Resampling.LANCZOS)
    ax.imshow(small_image)

for ax in axes.flat[len(prompts):]:
    ax.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# ! accelerate launch diffusers/examples/dreambooth/train_dreambooth_lora_sd3.py \
#   --pretrained_model_name_or_path="stabilityai/stable-diffusion-3.5-large"  \
#   --dataset_name="./dataset/" \
#   --output_dir="tuned-model-3.5-large-lora" \
#   --resolution=256 \
#   --train_batch_size=1 \
#   --gradient_accumulation_steps=1 \
#   --learning_rate=1e-4 \
#   --lr_scheduler="cosine" \
#   --lr_warmup_steps=100 \
#   --seed="0" \
#   --image_column="image" \
#   --caption_column="text" \
#   --mixed_precision="fp16" \
#   --use_8bit_adam \
#   --max_train_steps=20000 \
#   --checkpointing_steps=5000 

In [ ]:
# from diffusers import StableDiffusion3Pipeline
# import torch


# model_path = "tuned-model-3.5-large-lora"
# model_name = "stabilityai/stable-diffusion-3.5-large"

# pipe_tuned = StableDiffusion3Pipeline.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")
# pipe_tuned.load_lora_weights(model_path)

In [ ]:
# import matplotlib.pyplot as plt
# from PIL import Image

# desired_height = 768
# desired_width = 768
# images_per_row = 3

# fig, axes = plt.subplots(
#     nrows=(len(prompts) + images_per_row - 1) // images_per_row,  # Расчёт строк
#     ncols=images_per_row,
#     figsize=(15, 15)
# )

# for i, prompt in enumerate(prompts):
#     ax = axes[i // images_per_row, i % images_per_row]  # Выбираем позицию
#     ax.axis("off")  # Убираем оси
#     ax.set_title(f"Prompt {i + 1}")

#     tuned_image = pipe_tuned(
#         prompt=prompt,
#         num_inference_steps=100,
#         guidance_scale=6.5,
#         height=desired_height,
#         width=desired_width
#     ).images[0]

#     small_image = tuned_image.resize((256, 256), Image.Resampling.LANCZOS)
#     ax.imshow(small_image)

# for ax in axes.flat[len(prompts):]:
#     ax.axis("off")

# plt.tight_layout()
# plt.show()


# Flux 1-dev: Fine tuning

In [ ]:
# !pip install -r diffusers/examples/flux-control/requirements.txt

accelerate launch  diffusers/examples/flux-control/train_control_flux.py \
  --pretrained_model_name_or_path="black-forest-labs/FLUX.1-dev" \
  --dataset_name="raulc0399/open_pose_controlnet" \
  --output_dir="tuned-model-flux" \
  --mixed_precision="f16" \
  --train_batch_size=2 \
  --dataloader_num_workers=4 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --proportion_empty_prompts=0.2 \
  --learning_rate=5e-5 \
  --adam_weight_decay=1e-4 \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=1000 \
  --checkpointing_steps=1000 \
  --max_train_steps=10000 \
  --offload \
  --seed="0" \

In [ ]:
# from diffusers import FluxControlPipeline, FluxTransformer2DModel
# import torch

# # Инициализация модели FLUX
# # transformer = FluxTransformer2DModel.from_pretrained("tuned-model-flux")  
# transformer = FluxTransformer2DModel.from_pretrained("black-forest-labs/FLUX.1-dev") 
# pipe_tuned = FluxControlPipeline.from_pretrained(
#     "black-forest-labs/FLUX.1-dev", 
#     transformer=transformer, 
#     torch_dtype=torch.float16
# ).to("cuda")


In [ ]:
# import matplotlib.pyplot as plt
# from PIL import Image

# desired_height = 768
# desired_width = 768
# images_per_row = 3

# fig, axes = plt.subplots(
#     nrows=(len(prompts) + images_per_row - 1) // images_per_row,  # Расчёт строк
#     ncols=images_per_row,
#     figsize=(15, 15)
# )

# for i, prompt in enumerate(prompts):
#     ax = axes[i // images_per_row, i % images_per_row]  # Выбираем позицию
#     ax.axis("off")  # Убираем оси
#     ax.set_title(f"Prompt {i + 1}")

#     tuned_image = pipe_tuned(
#         prompt=prompt,
#         num_inference_steps=70,
#         guidance_scale=6.5,
#         random_state=42,
#         height=desired_height,
#         width=desired_width
#     ).images[0]

#     small_image = tuned_image.resize((256, 256), Image.Resampling.LANCZOS)
#     ax.imshow(small_image)

# for ax in axes.flat[len(prompts):]:
#     ax.axis("off")

# plt.tight_layout()
# plt.show()
